In [3]:
pwd

'/mmfs1/data/bergerar/BC-MRI-ADHD/Code'

In [4]:
import os 
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from make_models import get_MRI_VAE_3D,get_MRI_CVAE_3D
from tqdm import tqdm 

2022-06-08 12:37:21.864455: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/local/apps/gcc/9.2.0/lib:/cm/local/apps/gcc/9.2.0/lib64:/cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2022-06-08 12:37:21.864478: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
ar = np.load('../Assets/brain_array-440.npz')
print(list(ar.keys()))

data = ar['data']
control = ar['controls']
patients = ar['patients']
n = data.shape[0]

print(n)
print(control)

['data', 'controls', 'patients']
440
[ True False  True  True False  True  True False False False  True  True
  True  True  True  True  True False  True  True  True  True False  True
  True  True False False  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False False  True
  True  True  True  True  True  True False  True  True  True  True  True
  True  True False  True  True  True False False False False False  True
  True  True False  True  True False  True  True  True False  True  True
 False False False  True False False False  True  True  True False False
 False False  True  True  True False False  True  True  True False  True
 False  True False False False  True False False False False False False
 False  True  True False False False False  True False False  True False
  True  True False False  True False  True False False False  True False
 False False  True False  True False  True  True False  True  True False
 False False  

In [7]:
data_patients = data[patients,:,:,:]
print(data_patients.shape)
data_control = data[control,:,:,:]
print(data_control.shape)

(179, 64, 64, 64)
(261, 64, 64, 64)


In [8]:
type(data_patients)

numpy.ndarray

In [16]:
class bergeronADHD200_data_loader:
    
    def __init__(self):
        self.batch_id = 0 #count batches

    def get_batch(self):
        self.adhd_data_batch = np.random.rand(32,64,64,64)
        self.td_data_batch = np.random.rand(32,64,64,64)
        
        self.batch_id += 1 # count batches 
        return self.adhd_data_batch,self.td_data_batch
    
    def __init__(self,data_dir,batch_size=32):
        
        self.data = data[data]
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.n_batches = int(self.data.shape[0]/self.batch_size)-1
        self.e = -1
        self.b = -1
        
        self.new_epoch()
        
    def new_epoch(self):
        self.e+=1
        self.perm_idx = np.random.permutation(np.arange(self.data.shape[0]))
        self.data = self.data[self.perm_idx,:,:,:]
        
#patient batch
    def get_patient_batch(self):
        
        if self.b==self.n_batches:
            self.new_epoch()
            self.b = -1
                    
        self.b+=1        
        self.batch_idx = np.arange(self.b*self.batch_size,self.b*self.batch_size+self.batch_size)
        self.batch = self.data[self.batch_idx,:,:,:]
                    
        return self.batch
    
#control batch
    
    def get_control_batch(self):
        
        if self.b==self.n_batches:
            self.new_epoch()
            self.b = -1
                    
        self.b+=1        
        self.batch_idx = np.arange(self.b*self.batch_size,self.b*self.batch_size+self.batch_size)
        self.batch = self.data[self.batch_idx,:,:,:]
                    
        return self.batch

In [17]:
loaderhehe = bergeronADHD200_data_loader(data_dir=data,batch_size=32)

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
loaderhehe.data.shape

In [ ]:
data = ar['data']
control = ar['controls']
patients = ar['patients']

In [ ]:
n = loaderhehe.data.shape[0]
print(n)

In [ ]:
batch_size = 32

In [ ]:
data_batch = ar[np.random.randint(low=0,high=n,size=batch_size)]

In [ ]:
data_batch_patients = patients[np.random.randint(low=0,high=n,size=batch_size),:,:,:]
data_batch_control = control[np.random.randint(low=0,high=n,size=batch_size),:,:,:]

In [ ]:
data_batch_control = data_stuff[np.random.randint(low=0,high=n,size=batch_size),:,:,:]

In [ ]:
loaderhehe.get_patient_batch();
loaderhehe.get_control_batch();

In [ ]:
cvae, z_encoder, s_encoder, cvae_decoder = get_MRI_CVAE_3D(input_shape=(64,64,64,1),
                                            latent_dim=2,
                                            beta=1,
                                            disentangle=False,
                                            gamma=1,
                                            bias=True,
                                            batch_size = 64,
                                            kernel_size = 3,
                                            filters = 32,
                                            intermediate_dim = 128,
                                            opt=None)